#### Generate list of panels and get the forecasts using different methods

In [1]:
import numpy as np
import pandas as pd
import forecast
import warnings
warnings.filterwarnings('ignore')
import time
from calcMAPE import calcMAPE

DELIM = ","

In [2]:
dataloc = "/home/tbrownex/data/Hackett/archive/OutFront/testData/"

In [3]:
train = pd.read_csv(dataloc+"train.csv", nrows=160000)
test  = pd.read_csv(dataloc+"test.csv",  nrows=80000)

# Convert their date format to a standard date
train["date"] = pd.to_datetime(train["date"])
test["date"]  = pd.to_datetime(test["date"])

In [4]:
outlier  = open(dataloc+"outliers.csv", "w")

In [5]:
train = train.set_index("panel")
test  = test.set_index("panel")
grp   = train.groupby(level=0)

In [7]:
#nonFeatures = ["date", "hour", "population"]

panelList = []
datesList = []
hoursList = []

errors   = []
preds    = []
actuals  = []

start = time.time()
for panel, data in grp:
    tst = test.loc[panel]
    # These are simplistic forecasters: just use the Time Block
    meanMorning   = data.loc[data["block_morning"]  ==1]["population"].mean()
    meanAfternoon = data.loc[data["block_afternoon"]==1]["population"].mean()
    meanEvening   = data.loc[data["block_evening"]  ==1]["population"].mean()
    
    #features = [x for x in data.columns if x not in nonFeatures]
    trainX = data[features]
    testX  = tst[features]
    trainX = np.array(trainX)
    testX  = np.array(testX)
    
    trainY = data["population"]
    testY  = tst["population"]
        
    d = {}                 # holds the data to pass to the forecasters
    d["trainX"] = trainX
    d["trainY"] = trainY
    d["testX"]  = testX
    d["testY"]  = testY
        
    predictions = forecast.LSTM(d)
    errorRate   = calcMAPE(predictions, testY)
    print(errorRate)
    input()
    # A very high error rate usually means the data is messed up
    if errorRate < 4:
        errors.append(errorRate)
        preds.append(predictions)
            
        panelList.append(panel)
        datesList.append(test["date"].reset_index(drop=True))
        hoursList.append(test["hour"].reset_index(drop=True))
        actuals.append(testY)
    else:
        rec = panel + "\n"
        outlier.write(rec)
        
outlier.close()
end = time.time()
print("Processing time: {:.0f} minutes".format((end-start)/60))

(8736,)


KeyboardInterrupt: 

In [ ]:
# Summary has each panel and the errors associated with each algo
with open(dataloc+"summary.csv", "w") as output:
    hdr = ["panel","LSTM"]
    hdr = ",".join(hdr) + "\n"
    output.write(hdr)
    errs = [list(a) for a in zip(panelList, errors, naiveerrors)]

    for x in errs:
        rec = ",".join(map(str, x)) +"\n"
        output.write(rec)

In [ ]:
# "Predictions" has each panel and the predictions associated with each algo
with open(dataloc+"predictions.csv", "w") as output:
    hdr = ["panel","date", "hour","actual","LSTM"]
    hdr = ",".join(hdr) + "\n"
    output.write(hdr)
    for x, p in enumerate(panelList):
        
        preds = [list(a) for a in zip(datesList[x], hoursList[x], actuals[x], RFpreds[x], STLpreds[x], XGBpreds[x])]
        for y in preds:
            rec = p +DELIM+DELIM.join(map(str, y)) +"\n"
            output.write(rec)